In [1]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import random
import tensorflow as tf
import numpy as np
import timeit
from sklearn import metrics
import scipy.io as sio
import csv
import os
import math 
import random as rand
from numpy import genfromtxt
from numpy import sum,isrealobj,sqrt
from numpy.random import standard_normal
import string

#import torch
#from torch import nn

import warnings
from scipy.spatial import distance


from importlib import reload
import sys
sys.path.append("../Imports/") # Adds higher directory to python modules path.

from Modules.Tools import MapTool
from Modules.Tools import BoxPlot
from Modules.Tools import Extractor
from Modules.Tools import peakCorrelation as pC
from Modules.Tools import ATmega2560_Instructions as Inst_Info
import time

# Notebook Parameters
- this is simply for obtaining the graphs with manual mapping. Pad is closely relate to the sampling rate for the EM signal.

In [2]:
save_folder = ""
load_signal_folder = "../Datasets/Signals/Numpy/segment_good/"
load_code_folder = "../Datasets/Instruction_Code/"
pad = 7
y_low=-2.8
y_high=2.8
amount=3200

# Load Program Code

In [3]:
Base_Benign_Code = np.load(load_code_folder + "Base_Benign.npy").tolist()
Base_Anom_Code = np.load(load_code_folder + "Base_Anom.npy").tolist()

In [4]:
Update_50_A_Benign_Code = np.load(load_code_folder + "Update_50_A_Benign.npy").tolist()
Update_50_B_Benign_Code = np.load(load_code_folder + "Update_50_B_Benign.npy").tolist()
Update_50_C_Benign_Code = np.load(load_code_folder + "Update_50_C_Benign.npy").tolist()

Update_50_A_Anom_Code = np.load(load_code_folder + "Update_50_A_Anom.npy").tolist()
Update_50_B_Anom_Code = np.load(load_code_folder + "Update_50_B_Anom.npy").tolist()
Update_50_C_Anom_Code = np.load(load_code_folder + "Update_50_C_Anom.npy").tolist()

In [5]:
Update_75_A_Benign_Code = np.load(load_code_folder + "Update_75_A_Benign.npy").tolist()
Update_75_B_Benign_Code = np.load(load_code_folder + "Update_75_B_Benign.npy").tolist()
Update_75_C_Benign_Code = np.load(load_code_folder + "Update_75_C_Benign.npy").tolist()

Update_75_A_Anom_Code = np.load(load_code_folder + "Update_75_A_Anom.npy").tolist()
Update_75_B_Anom_Code = np.load(load_code_folder + "Update_75_B_Anom.npy").tolist()
Update_75_C_Anom_Code = np.load(load_code_folder + "Update_75_C_Anom.npy").tolist()

In [6]:
Update_100_A_Benign_Code = np.load(load_code_folder + "Update_100_A_Benign.npy").tolist()
Update_100_B_Benign_Code = np.load(load_code_folder + "Update_100_B_Benign.npy").tolist()
Update_100_C_Benign_Code = np.load(load_code_folder + "Update_100_C_Benign.npy").tolist()

Update_100_A_Anom_Code = np.load(load_code_folder + "Update_100_A_Anom.npy").tolist()
Update_100_B_Anom_Code = np.load(load_code_folder + "Update_100_B_Anom.npy").tolist()
Update_100_C_Anom_Code = np.load(load_code_folder + "Update_100_C_Anom.npy").tolist()

In [7]:
Train_Program_1_Code = np.load(load_code_folder + "Train_Program_1.npy").tolist()
Train_Program_2_Code = np.load(load_code_folder + "Train_Program_2.npy").tolist()
Train_Program_3_Code = np.load(load_code_folder + "Train_Program_3.npy").tolist()
Train_Program_4_Code = np.load(load_code_folder + "Train_Program_4.npy").tolist()

# Load Signals

In [8]:
Base_Benign = np.load(load_signal_folder + "Base_Benign.npy", allow_pickle=True)
Base_Anom = np.load(load_signal_folder + "Base_Anom.npy", allow_pickle=True)

In [9]:
Update_50_A_Benign = np.load(load_signal_folder + "Update_50_A_Benign.npy", allow_pickle=True)
Update_50_B_Benign = np.load(load_signal_folder + "Update_50_B_Benign.npy", allow_pickle=True)
Update_50_C_Benign = np.load(load_signal_folder + "Update_50_C_Benign.npy", allow_pickle=True)

Update_50_A_Anom = np.load(load_signal_folder + "Update_50_A_Anom.npy", allow_pickle=True)
Update_50_B_Anom = np.load(load_signal_folder + "Update_50_B_Anom.npy", allow_pickle=True)
Update_50_C_Anom = np.load(load_signal_folder + "Update_50_C_Anom.npy", allow_pickle=True)

In [10]:
Update_75_A_Benign = np.load(load_signal_folder + "Update_75_A_Benign.npy", allow_pickle=True)
Update_75_B_Benign = np.load(load_signal_folder + "Update_75_B_Benign.npy", allow_pickle=True)
Update_75_C_Benign = np.load(load_signal_folder + "Update_75_C_Benign.npy", allow_pickle=True)

Update_75_A_Anom = np.load(load_signal_folder + "Update_75_A_Anom.npy", allow_pickle=True)
Update_75_B_Anom = np.load(load_signal_folder + "Update_75_B_Anom.npy", allow_pickle=True)
Update_75_C_Anom = np.load(load_signal_folder + "Update_75_C_Anom.npy", allow_pickle=True)

In [11]:
Update_100_A_Benign = np.load(load_signal_folder + "Update_100_A_Benign.npy", allow_pickle=True)
Update_100_B_Benign = np.load(load_signal_folder + "Update_100_B_Benign.npy", allow_pickle=True)
Update_100_C_Benign = np.load(load_signal_folder + "Update_100_C_Benign.npy", allow_pickle=True)

Update_100_A_Anom = np.load(load_signal_folder + "Update_100_A_Anom.npy", allow_pickle=True)
Update_100_B_Anom = np.load(load_signal_folder + "Update_100_B_Anom.npy", allow_pickle=True)
Update_100_C_Anom = np.load(load_signal_folder + "Update_100_C_Anom.npy", allow_pickle=True)

In [12]:
Train_Program_1 = np.load(load_signal_folder + "Train_Program_1.npy", allow_pickle=True)
Train_Program_2 = np.load(load_signal_folder + "Train_Program_2.npy", allow_pickle=True)
Train_Program_3 = np.load(load_signal_folder + "Train_Program_3.npy", allow_pickle=True)
Train_Program_4 = np.load(load_signal_folder + "Train_Program_4.npy", allow_pickle=True)

# Next,  create training datasets based on the extracted information and the signals of the sequences. I.E. list of the following.

- Prior and target instructions (Sequence)
- Register Values for each instruction in the Sequence
- State before exeucting the target instruction.
- Signals of set size.
    - Signals will be of the smallest length of in the middle (0) point value to cycle length for each instruction in the sequence.
    - What this incorporates is to take into acount the amount length of each instructions signal, remove from each end till the size, and stitch/append/combine together into a set size.

In [13]:
# Load the extracted information from the asm program code that was saved in 2_Extract_Important_Information
train_Set_Extracted = np.load("../Datasets/Extracted Information/train_Set_Extracted.npy", allow_pickle=True)

In [14]:
'''
Given lists of ASM_Inst, c_Values,  states of each Training Set in a list and a list of numpy arrays of sequence signals,

Input:
    train_Set_Extracted: list of ASM_Inst, list of c_Values, list of states of each Train_Program in that order.
    train_Set_Signals: list of numpy arrays of sequence signals. (must be in the same order as Train_Set_Extracted)
    sequence_size: sequence size of the number of instructions (prior and target) to consider.

Output:
    Sequence_Code: list of sequence code for each program executed in order of given Training_Set_Extracted.
    Register_Values: list of register values for each instruction in the each sequence for each program executed in order of give Training_Set_Extracted.
    Target_States: list of the state prior to executing the target instruction.
    Signal_Priors: list of signals corresponding to the prior instructions. 
        (Note: will be prior + 1 = same as sequence_size as Transformers need the prior signal to generate a new signal of same shape.
            This new signal will include some of the prior signal with added new generated time index values/amplitudes)
'''
def extractInformationFromTrainingSets(train_Set_Extracted, sequence_size, include_c_Values=True, include_States=True):
    x_repeats = []
    y_repeats = []
    
    # First obtain the Sequence_Code, Register_Values and Target_States
    ## cycle through each training set
    for train_Set in train_Set_Extracted:
        ## Obtain the assembly instructions (ASM_Inst), current values (c_Values), and states.
        ASM_Inst, c_Values, states = train_Set
        
        ## Cycle though each instance
        for i in range(len(ASM_Inst)-sequence_size):
            info=[]
            target = ""
            ## Cycle through sequence_size
            for n in range(sequence_size+1):
                ## obtain the instruction in the sequence
                info.append(ASM_Inst[i+n])
                target = ASM_Inst[i+n]

                ## obtain current values before executing each instruction in the sequence
                if include_c_Values:
                    info.append(c_Values[i-n][0])
                    info.append(c_Values[i-n][1])
                    
            ## Add the state before executing the target instruction
            if include_States:
                info = info + states[i+sequence_size]
                
            ## Since the model generates cycles (2 waves) at a time, we need to check how many cycles the target instruction
            ## has and also add the info for each cycle along with an indicator on which cycle to generate.
            #print(target)
            #print(Inst_Info.getCycles(target))
            amount_Cycles = Inst_Info.getCycles(target)
            for i in range(amount_Cycles):
                info_cycle = info.copy()
                info_cycle.append(i+1)
                x_repeats.append(info_cycle)
                
        print(len(x_repeats))
            
    return x_repeats

In [15]:
temp_0 = extractInformationFromTrainingSets(train_Set_Extracted, sequence_size=0)
temp_1 = extractInformationFromTrainingSets(train_Set_Extracted, sequence_size=1)
temp_2 = extractInformationFromTrainingSets(train_Set_Extracted, sequence_size=2)

183
379
553
688
182
377
550
684
180
374
545
678


In [16]:
print(temp_0[0:10])

[['inc', 1, -1, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 1, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0, 1], ['rjmp', -1, -1, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0, 1], ['rjmp', -1, -1, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0, 2], ['cp', 4, 10, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0, 1], ['brlo-t', -1, -1, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0, 1], ['brlo-t', -1, -1, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0, 2], ['cpi', 24, 20, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0, 1], ['brlo-f', -1, -1, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0, 1], ['add', 0, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0

In [17]:
print(temp_1[0:10])

[['inc', 1, -1, 'rjmp', -1, -1, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0, 1], ['inc', 1, -1, 'rjmp', -1, -1, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0, 2], ['rjmp', -1, -1, 'cp', 1, -1, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0, 1], ['cp', 4, 10, 'brlo-t', -1, -1, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0, 1], ['cp', 4, 10, 'brlo-t', -1, -1, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0, 2], ['brlo-t', -1, -1, 'cpi', 4, 10, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0, 1], ['cpi', 24, 20, 'brlo-f', -1, -1, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0, 1], ['brlo-f', -1, -1, 'add', 24, 20, 6, 0, 0, 0, 0, 0, 0, 

In [18]:
print(temp_2[0:10])

[['inc', 1, -1, 'rjmp', -1, -1, 'cp', 0, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0, 1], ['rjmp', -1, -1, 'cp', 1, -1, 'brlo-t', -1, -1, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0, 1], ['rjmp', -1, -1, 'cp', 1, -1, 'brlo-t', -1, -1, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0, 2], ['cp', 4, 10, 'brlo-t', -1, -1, 'cpi', 1, -1, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0, 1], ['brlo-t', -1, -1, 'cpi', 4, 10, 'brlo-f', -1, -1, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0, 1], ['cpi', 24, 20, 'brlo-f', -1, -1, 'add', 4, 10, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0, 1], ['brlo-f', -1, -1, 'add', 24, 20, 'adc', -1, -1, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [19]:
print(train_Set_Extracted[0][0])
print(len(train_Set_Extracted[0][0]))

['inc', 'rjmp', 'cp', 'brlo-t', 'cpi', 'brlo-f', 'add', 'adc', 'sub', 'sbc', 'mov', 'rjmp', 'clr', 'clr', 'ldi', 'clr', 'ldi', 'cp', 'brlo-f', 'sub', 'inc', 'cpi', 'brlo-f', 'mul', 'mov', 'mov', 'mul', 'mov', 'clr', 'inc', 'clr', 'ldi', 'clr', 'ldi', 'clr', 'clr', 'clr', 'mul', 'mov', 'rjmp', 'ldi', 'ldi', 'ldi', 'mul', 'add', 'adc', 'sub', 'sbc', 'mov', 'rjmp', 'clr', 'clr', 'clr', 'ldi', 'ldi', 'ldi', 'mul', 'clr', 'clr', 'clr', 'clr', 'clr', 'mov', 'mov', 'add', 'brcc-t', 'clr', 'clr', 'clr', 'mul', 'mov', 'rjmp', 'clr', 'clr', 'clr', 'mul', 'mov', 'rjmp', 'inc', 'clr', 'ldi', 'ldi', 'ldi', 'mul', 'mov', 'sub', 'brcc-t', 'mov', 'rjmp', 'ldi', 'ldi', 'ldi', 'ldi', 'clr', 'clr', 'clr', 'mul', 'mov', 'rjmp', 'clr', 'clr', 'ldi', 'ldi', 'ldi', 'ldi', 'mul', 'clr', 'clr', 'mov', 'mov', 'add', 'brcc-t', 'cpi', 'brlo-f', 'mul', 'mov', 'cpi', 'brlo-t', 'mul', 'mov', 'mov', 'mul', 'mov', 'clr', 'mul', 'clr', 'mul', 'mov', 'mul', 'inc', 'ldi', 'ldi', 'mov', 'mul', 'mov', 'clr', 'mul', 'add', 

In [20]:
print(train_Set_Extracted[0][1])
print(len(train_Set_Extracted[0][1]))

[[1, -1], [-1, -1], [4, 10], [-1, -1], [24, 20], [-1, -1], [0, 6], [4, 0], [0, 6], [10, 4], [2, 250], [-1, -1], [250, -1], [250, -1], [5, 10], [6, -1], [4, 24], [24, 10], [-1, -1], [24, 10], [0, -1], [24, 20], [-1, -1], [4, 24], [0, 96], [10, 0], [2, 2], [0, 4], [14, -1], [1, -1], [96, -1], [0, 10], [4, -1], [0, 4], [4, -1], [0, -1], [10, -1], [2, 3], [0, 6], [-1, -1], [3, 83], [4, 139], [2, 200], [83, 83], [0, 233], [4, 26], [0, 233], [6, 30], [2, 23], [-1, -1], [233, -1], [26, -1], [231, -1], [200, 2], [83, 3], [139, 4], [2, 3], [23, -1], [23, -1], [233, -1], [30, -1], [10, -1], [0, 6], [0, 0], [6, 4], [-1, -1], [6, -1], [0, -1], [10, -1], [2, 3], [0, 6], [-1, -1], [6, -1], [0, -1], [6, -1], [2, 3], [0, 6], [-1, -1], [0, -1], [6, -1], [0, 6], [0, 14], [0, 10], [4, 24], [14, 96], [96, 6], [-1, -1], [0, 90], [-1, -1], [90, 48], [3, 225], [4, 18], [2, 9], [96, -1], [10, -1], [1, -1], [9, 24], [6, 216], [-1, -1], [48, -1], [0, -1], [9, 2], [225, 3], [18, 4], [0, 0], [2, 3], [216, -1], [9

In [21]:
print(train_Set_Extracted[0][2])
print(len(train_Set_Extracted[0][2]))

[[6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 1, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0], [6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0], [6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0], [6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0], [6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0], [6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0], [6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 0, 4, 10, 0, 0, 0, 0, 0, 0], [6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 6, 4, 10, 0, 0, 0, 0, 0, 0], [6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 0, 10, 6, 4, 10, 0, 0, 0, 0, 0, 0], [6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 24, 2, 250, 10, 6, 4, 10, 0, 0, 

## This method simply calculates the minimum and maximum index (time) 1 cycle (bottom of prior cycle to top of the next cycle) and to top takes across all training program in order to make an informed choice of the necessary length of the cycle for the model to generate.
- the cycles may take slightly different index length due to cycle/time drifting.

In [23]:
'''
Returns all unique lengths of the time (indexies) it takes to complete one cycle
'''
def getLengthCycles(train_Set_Extracted, train_Set_Signals, train_Set_Codes, sequence_size, 
                                       include_c_Values=True, include_States=True, pad = 7, check=0):
    start_peak = -2
    end_peak = 1
    
    # First obtain the Sequence_Code, Register_Values and Target_States
    ## cycle through each training set
    train_Set = train_Set_Extracted[check]
    train_Signals = train_Set_Signals[check]
    train_Code = train_Set_Codes[check]

    ## Obtain the assembly instructions (ASM_Inst), current values (c_Values), and states.
    ASM_Inst, c_Values, states = train_Set

    ## Cycle though each instance
    for i in range(len(ASM_Inst)-sequence_size):
        info=[]
        target = ""

        ## Cycle through sequence_size
        for n in range(sequence_size+1):
            ## obtain the instruction in the sequence
            info.append(ASM_Inst[i+n])
            target = ASM_Inst[i+n]

            ## obtain current values before executing each instruction in the sequence
            if include_c_Values:
                info.append(c_Values[i-n][0])
                info.append(c_Values[i-n][1])

        ## Since the model generates cycles (2 waves) at a time, we need to check how many cycles the target instruction
        ## has and also add the info for each cycle along with an indicator on which cycle to generate.
        #print(target)
        #print(Inst_Info.getCycles(target))
        amount_Cycles = Inst_Info.getCycles(target)
        for i in range(amount_Cycles):

            # Add to the y instance of the target instructions cyles  signal
            ## Identify the start and end of the signal peaks to extract for y
            start_peak = start_peak + 2
            end_peak = end_peak + 2

            # Checker
            ## Check if taking correct from full signal
            '''
            code = train_Code
            Signal = train_Signals[0]

            time_Signal = np.arange(Signal.shape[0])
            plt.plot(time_Signal, Signal)
            MapTool.mapCodeSignal(Signal, code, pad,skip_start_peaks=1)
            plt.axvspan(MapTool.getPeaksLoc(Signal, pad, start_peak, getHigh = False), MapTool.getPeaksLoc(Signal, pad, end_peak), color='orange', alpha=0.5)
            plt.xlabel("Sample Index")
            plt.ylabel("Amplitude")
            plt.grid(axis='y')

            plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12),
                      fancybox=True, shadow=True, ncol=6)
            plt.xlim([0,len(Signal)])
            plt.ylim(y_low,y_high)
            plt.show()
            plt.close()
            '''

            ## Zoom in to area
            '''
            code = train_Code
            Signal = train_Signals[0]

            time_Signal = np.arange(Signal.shape[0])
            plt.plot(time_Signal, Signal)
            MapTool.mapCodeSignal(Signal, code, pad,skip_start_peaks=1)
            plt.axvspan(MapTool.getPeaksLoc(Signal, pad, start_peak,getHigh = False), MapTool.getPeaksLoc(Signal, pad, end_peak), color='orange', alpha=0.5)
            plt.xlabel("Sample Index")
            plt.ylabel("Amplitude")
            plt.grid(axis='y')

            plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12),
                      fancybox=True, shadow=True, ncol=6)
            plt.xlim([MapTool.getPeaksLoc(Signal, pad, start_peak,getHigh = False)-5,MapTool.getPeaksLoc(Signal, pad, end_peak)+5])
            plt.ylim(y_low,y_high)
            plt.show()
            plt.close()
            '''

            ## Check if taking correct from Cycle obtained
            Cycles = pC.getSeqSignals(train_Signals, start_peak, end_peak, startHigh=False, pad=pad)

            lengths = []
            for cycle in Cycles:
                lengths.append(len(cycle))

            lengths = np.array(lengths)
            print(np.unique(lengths))

            '''
            #code = [target]
            Signal = Cycles[0]

            time_Signal = np.arange(Signal.shape[0])
            plt.plot(time_Signal, Signal)
            #MapTool.mapCodeSignal(Signal, code, pad)
            #plt.axvspan(MapTool.getPeaksLoc(Signal, pad, start_peak,getHigh = False), MapTool.getPeaksLoc(Signal, pad, end_peak), color='orange', alpha=0.5)
            plt.xlabel("Sample Index")
            plt.ylabel("Amplitude")
            plt.grid(axis='y')

            #plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12),
            #          fancybox=True, shadow=True, ncol=6)
            plt.xlim([0,len(Signal)])
            plt.ylim(y_low,y_high)
            plt.show()
            plt.close()
            '''

In [21]:
getLengthCycles(train_Set_Extracted, train_Set_Signals, train_Set_Codes,sequence_size=0, pad = pad, check=1)

[38 39 40 41]
[38 39 40 41]
[38 39 40 41]
[38 39 40 41]
[38 39 40 41]
[38 39 40 41]
[38 39 40 41]
[38 39 40 41]
[38 39 40 41]
[38 39 40 41]
[38 39 40 41]
[38 40 41]
[38 39 40 41]
[38 39 40 41]
[38 39 40 41]
[38 39 40 41]
[38 39 40 41]
[38 39 40 41]
[38 39 40 41]
[38 39 40 41]
[38 39 40 41]
[38 39 40 41]
[38 39 40 41]
[38 40 41]
[38 39 40 41]
[38 39 40 41]
[38 39 40 41]
[38 39 40 41]
[38 39 40 41]
[37 38 39 40 41]
[38 39 40 41]
[38 39 40 41]
[37 38 39 40 41]
[38 39 40 41]
[38 39 40 41]
[38 39 40 41]
[38 39 40 41]
[38 39 40 41]
[37 38 39 40 41]


KeyboardInterrupt: 

# most cycles are 38 or more. Thus, a check if the cycle is 38 or more is done and take that as an instance to save. Additionally, additional information is removed till it has 38 index size.

In [24]:
def cutCycleToLength(cycle, set_length):
    if len(cycle) < set_length:
        print("length of cycle: ", len(cycle))
        Signal = cycle

        time_Signal = np.arange(Signal.shape[0])
        plt.plot(time_Signal, Signal)
        plt.xlabel("Sample Index")
        plt.ylabel("Amplitude")
        plt.grid(axis='y')

        plt.xlim([0,len(Signal)])
        plt.ylim(y_low,y_high)
        plt.show()
        plt.close()
        raise ValueError("set_length is larger than the cycle length!")

    # Calculate the amount to remove and split evenly between start and end
    amount_to_remove = len(cycle) - set_length
    start_remove = amount_to_remove // 2
    end_remove = amount_to_remove - start_remove
    
    stop = 0
    if len(cycle) == set_length:
        stop = 1
    while stop == 0:
        if cycle[len(cycle)-end_remove] < 0 and cycle[start_remove +1] < 0:
            end_remove = end_remove + 1
            start_remove = start_remove + 1
        elif cycle[len(cycle)-end_remove] >= 0:
            stop = 1
        else:
            print("End cycle point: ", cycle[len(cycle)-end_remove])
            print("Would be start cycle point: ", cycle[start_remove +1])
            raise ValueError("Cannot remove more information!")

    # Slice the cycle directly
    return cycle[start_remove:len(cycle) - end_remove]

In [25]:
def cutCyclesToLength(dataset, set_length):
    new_dataset = []
    for cycle in dataset:
        if len(cycle) >= set_length:
            new_dataset.append(cutCycleToLength(cycle, set_length))
    return new_dataset

# Create a method to extract the y (signal cycle for each instance)

In [26]:
'''
Given lists of ASM_Inst, c_Values,  states of each Training Set in a list and a list of numpy arrays of sequence signals,

Input:
    train_Set_Extracted: list of ASM_Inst, list of c_Values, list of states of each Train_Program in that order.
    train_Set_Signals: list of numpy arrays of sequence signals. (must be in the same order as Train_Set_Extracted)
    sequence_size: sequence size of the number of instructions (prior instructions only) to consider.

Output:
    Sequence_Code: list of sequence code for each program executed in order of given Training_Set_Extracted.
    Register_Values: list of register values for each instruction in the each sequence for each program executed in order of give Training_Set_Extracted.
    Target_States: list of the state prior to executing the target instruction.
    Signal_Priors: list of signals corresponding to the prior instructions. 
        (Note: will be prior + 1 = same as sequence_size as Transformers need the prior signal to generate a new signal of same shape.
            This new signal will include some of the prior signal with added new generated time index values/amplitudes)
'''
def extractInformationFromTrainingSets(train_Set_Extracted, train_Set_Signals, train_Set_Codes, sequence_size, 
                                       include_c_Values=True, include_States=True, pad = 7, set_length = 35, amount = 3000):
    x_repeats = []
    y_repeats = []
    start_peak = -2
    end_peak = 1
    
    # First obtain the Sequence_Code, Register_Values and Target_States
    ## cycle through each training set
    for t in range(train_Set_Extracted.shape[0]):
        train_Set = train_Set_Extracted[t]
        train_Signals = train_Set_Signals[t]
        train_Code = train_Set_Codes[t]
        
        # Adjust how much to skip at the start for each cycle that is a prior instruction for first target instruction.
        for i in range(sequence_size):
            prior_inst = train_Code[i]
            amount_Cycles = Inst_Info.getCycles(prior_inst)
            for n in range(amount_Cycles):
                start_peak = start_peak + 2
                end_peak = end_peak +2
        
        ## Obtain the assembly instructions (ASM_Inst), current values (c_Values), and states.
        ASM_Inst, c_Values, states = train_Set
        
        ## Cycle though each instance
        for i in range(len(ASM_Inst)-sequence_size):
            info=[]
            target = ""
            ## Cycle through sequence_size
            for n in range(sequence_size+1):
                ## obtain the instruction in the sequence
                info.append(ASM_Inst[i+n])
                target = ASM_Inst[i+n]

                ## obtain current values before executing each instruction in the sequence
                if include_c_Values:
                    info.append(c_Values[i-n][0])
                    info.append(c_Values[i-n][1])
                    
            ## Add the state before executing the target instruction
            if include_States:
                info = info + states[i+sequence_size]
                
            ## Since the model generates cycles (2 waves) at a time, we need to check how many cycles the target instruction
            ## has and also add the info for each cycle along with an indicator on which cycle to generate.
            #print(target)
            #print(Inst_Info.getCycles(target))
            amount_Cycles = Inst_Info.getCycles(target)
            for i in range(amount_Cycles):
                info_cycle = info.copy()
                info_cycle.append(i+1)
                x_repeats.append(info_cycle)
                
        print(len(x_repeats))
            
    return x_repeats

In [27]:
train_Set_Signals = []
train_Set_Signals.append(Train_Program_1)
train_Set_Signals.append(Train_Program_2)
train_Set_Signals.append(Train_Program_3)
train_Set_Signals.append(Train_Program_4)
train_Set_Signals = np.array(train_Set_Signals, dtype=object)

In [28]:
train_Set_Codes = []
train_Set_Codes.append(Train_Program_1_Code)
train_Set_Codes.append(Train_Program_2_Code)
train_Set_Codes.append(Train_Program_3_Code)
train_Set_Codes.append(Train_Program_4_Code)
train_Set_Codes = np.array(train_Set_Codes, dtype=object)

In [29]:
x_0 = extractInformationFromTrainingSets(train_Set_Extracted, train_Set_Signals, train_Set_Codes,sequence_size=0, pad = pad)

183
379
553
688


In [30]:
x_1 = extractInformationFromTrainingSets(train_Set_Extracted, train_Set_Signals, train_Set_Codes,sequence_size=1, pad = pad)

182
377
550
684


In [31]:
x_2 = extractInformationFromTrainingSets(train_Set_Extracted, train_Set_Signals, train_Set_Codes,sequence_size=2, pad = pad)

180
374
545
678


# Problem is that y takes awhile to calculate.
- However this y will stay the same each time, just remove of the first few instances.
- Thus, we can calculate the y values for 0, then remove the first few for 1 and 2 prior instructions to calculate and save them.
- Then we can just use load the ones with what ever x values that are for the same sequence sizes.

In [32]:
import time

In [33]:
def getCorrect(Cycles, amount=2500, pad=8):
    new_Cycles = []
    Peaks_Cycles = MapTool.getPeaksDataset(Cycles, pad, getHigh = True)
    count = 0
    for c in range(len(Cycles)):
        peaks = Peaks_Cycles[c]
        if len(peaks) == 3:
            if peaks[0] > peaks[1]:
                new_Cycles.append(Cycles[c])
            else:
                count = count+1
    if count > 0:
        print("Number of off set cycles: ", count)
    return new_Cycles[:amount]

In [54]:
'''
Given lists of ASM_Inst, c_Values,  states of each Training Set in a list and a list of numpy arrays of sequence signals,

Input:
    train_Set_Extracted: list of ASM_Inst, list of c_Values, list of states of each Train_Program in that order.
    train_Set_Signals: list of numpy arrays of sequence signals. (must be in the same order as Train_Set_Extracted)
    sequence_size: sequence size of the number of instructions (prior instructions only) to consider.

Output:
    Sequence_Code: list of sequence code for each program executed in order of given Training_Set_Extracted.
    Register_Values: list of register values for each instruction in the each sequence for each program executed in order of give Training_Set_Extracted.
    Target_States: list of the state prior to executing the target instruction.
    Signal_Priors: list of signals corresponding to the prior instructions. 
        (Note: will be prior + 1 = same as sequence_size as Transformers need the prior signal to generate a new signal of same shape.
            This new signal will include some of the prior signal with added new generated time index values/amplitudes)
'''
def extractInformationFromTrainingSets_y(train_Set_Extracted, train_Set_Signals, train_Set_Codes, sequence_size, 
                                       include_c_Values=True, include_States=True, pad = 7, set_length = 38, amount = 2500, t=0):
    y_repeats = []
    count = 0
    start_time = time.time()
    
    # First obtain the Sequence_Code, Register_Values and Target_States
    ## cycle through each training set
    train_Set = train_Set_Extracted[t]
    train_Signals = train_Set_Signals[t]
    train_Code = train_Set_Codes[t]

    start_peak = -2
    end_peak = 1

    # Adjust how much to skip at the start for each cycle that is a prior instruction for first target instruction.
    for i in range(sequence_size):
        prior_inst = train_Code[i]
        amount_Cycles = Inst_Info.getCycles(prior_inst)
        for n in range(amount_Cycles):
            start_peak = start_peak + 2
            end_peak = end_peak +2

    ## Obtain the assembly instructions (ASM_Inst), current values (c_Values), and states.
    ASM_Inst, c_Values, states = train_Set

    ## Cycle though each instance
    for i in range(len(ASM_Inst)-sequence_size):
        info=[]
        target = ""
        ## Cycle through sequence_size
        for n in range(sequence_size+1):
            ## obtain the instruction in the sequence
            info.append(ASM_Inst[i+n])
            target = ASM_Inst[i+n]

        ## Since the model generates cycles (2 waves) at a time, we need to check how many cycles the target instruction
        ## has and also add the info for each cycle along with an indicator on which cycle to generate.
        #print(target)
        #print(Inst_Info.getCycles(target))
        amount_Cycles = Inst_Info.getCycles(target)
        for i in range(amount_Cycles):

            # Add to the y instance of the target instructions cyles  signal
            ## Identify the start and end of the signal peaks to extract for y
            start_peak = start_peak + 2
            end_peak = end_peak + 2

            # Obtain the cycle for the target instruction.
            ## Check if taking correct from full signal
            '''
            code = train_Code
            Signal = train_Signals[0]

            time_Signal = np.arange(Signal.shape[0])
            plt.plot(time_Signal, Signal)
            MapTool.mapCodeSignal(Signal, code, pad,skip_start_peaks=1)
            plt.axvspan(MapTool.getPeaksLoc(Signal, pad, start_peak, getHigh = False), MapTool.getPeaksLoc(Signal, pad, end_peak), color='orange', alpha=0.5)
            plt.xlabel("Sample Index")
            plt.ylabel("Amplitude")
            plt.grid(axis='y')

            plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12),
                      fancybox=True, shadow=True, ncol=6)
            plt.xlim([0,len(Signal)])
            plt.ylim(y_low,y_high)
            plt.show()
            plt.close()
            '''


            ## Zoom in to area
            '''
            code = train_Code
            Signal = train_Signals[0]

            time_Signal = np.arange(Signal.shape[0])
            plt.plot(time_Signal, Signal)
            MapTool.mapCodeSignal(Signal, code, pad,skip_start_peaks=1)
            plt.axvspan(MapTool.getPeaksLoc(Signal, pad, start_peak,getHigh = False), MapTool.getPeaksLoc(Signal, pad, end_peak), color='orange', alpha=0.5)
            plt.xlabel("Sample Index")
            plt.ylabel("Amplitude")
            plt.grid(axis='y')

            plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12),
                      fancybox=True, shadow=True, ncol=6)
            plt.xlim([MapTool.getPeaksLoc(Signal, pad, start_peak,getHigh = False)-5,MapTool.getPeaksLoc(Signal, pad, end_peak)+5])
            plt.ylim(y_low,y_high)
            plt.show()
            plt.close()
            '''


            # obtain the signal for the cycle
            Cycles = pC.getSeqSignals(train_Signals, start_peak, end_peak, startHigh=False, pad=pad)

            ## Check if taking correct from Cycle obtained
            '''
            #code = [target]
            Signal = Cycles[0]

            time_Signal = np.arange(Signal.shape[0])
            plt.plot(time_Signal, Signal)
            #MapTool.mapCodeSignal(Signal, code, pad)
            #plt.axvspan(MapTool.getPeaksLoc(Signal, pad, start_peak,getHigh = False), MapTool.getPeaksLoc(Signal, pad, end_peak), color='orange', alpha=0.5)
            plt.xlabel("Sample Index")
            plt.ylabel("Amplitude")
            plt.grid(axis='y')

            #plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12),
            #          fancybox=True, shadow=True, ncol=6)
            plt.xlim([0,len(Signal)])
            plt.ylim(y_low,y_high)
            plt.show()
            plt.close()
            '''


            ## Remove information to have same length cycles
            Cycles = cutCyclesToLength(Cycles, set_length)
            Cycles = getCorrect(Cycles, amount, pad)
            if len(Cycles) < amount:
                print("Cycles saved is lower than expected: amount ", len(Cycles))
                print("Count: ", count)
                
                Cycles = Cycles + Cycles
                Cycles = Cycles[:amount]
                if len(Cycles) < amount:
                    print("Still the Cycles saved is lower than expected: amount ", len(Cycles))
                    print("Count: ", count)
                    
                    Cycles = Cycles + Cycles
                    Cycles = Cycles[:amount]
                    if len(Cycles) < amount:
                        print("Warning To few of good samples: amount ", len(Cycles))
                        print("Count: ", count)

            '''
            #code = [target]
            Signal = Cycles[0]

            time_Signal = np.arange(Signal.shape[0])
            plt.plot(time_Signal, Signal)
            #MapTool.mapCodeSignal(Signal, code, pad)
            #plt.axvspan(MapTool.getPeaksLoc(Signal, pad, start_peak,getHigh = False), MapTool.getPeaksLoc(Signal, pad, end_peak), color='orange', alpha=0.5)
            plt.xlabel("Sample Index")
            plt.ylabel("Amplitude")
            plt.grid(axis='y')

            #plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12),
            #          fancybox=True, shadow=True, ncol=6)
            plt.xlim([0,len(Signal)])
            plt.ylim(y_low,y_high)
            plt.show()
            plt.close()
            '''

            count = count +1
            if count % 50 == 0 or count == 1:
                print("Count: ", count)
                temp = np.array(Cycles)
                print("Check shape: ", temp.shape)
                end_time = time.time()
                elapsed_time = end_time - start_time
                print(elapsed_time)

            y_repeats.append(Cycles)
    print(len(y_repeats))
            
    return y_repeats

In [44]:
list1 = [[2,3],[4,5],[6,7]]
list2 = [[2,3],[4,5],[6,7]]

temp = list1 + list2
print(temp)

[[2, 3], [4, 5], [6, 7], [2, 3], [4, 5], [6, 7]]


In [45]:
y_Train_Program_1 = extractInformationFromTrainingSets_y(train_Set_Extracted, train_Set_Signals, train_Set_Codes,sequence_size=0, pad = pad, t=0)

Count:  1
Check shape:  (2500, 38)
2.4140288829803467
Number of off set cycles:  127
Number of off set cycles:  113
Number of off set cycles:  3
Number of off set cycles:  33
Count:  50
Check shape:  (2500, 38)
318.3247265815735
Number of off set cycles:  384
Number of off set cycles:  83
Number of off set cycles:  33
Number of off set cycles:  4
Number of off set cycles:  180
Number of off set cycles:  201
Number of off set cycles:  1
Number of off set cycles:  640
Cycles saved is lower than expected: amount  2432
Count:  86
Number of off set cycles:  56
Count:  100
Check shape:  (2500, 38)
1079.79785490036
Number of off set cycles:  493
Number of off set cycles:  76
Number of off set cycles:  75
Number of off set cycles:  556
Number of off set cycles:  130
Number of off set cycles:  1666
Cycles saved is lower than expected: amount  1406
Count:  145
Count:  150
Check shape:  (2500, 38)
2277.0351972579956
Number of off set cycles:  234
Number of off set cycles:  227
183


In [51]:
np.save("../Datasets/Cycle Signals/y_Train_Program_1_No_Offset.npy", np.array(y_Train_Program_1))

In [55]:
y_Train_Program_2 = extractInformationFromTrainingSets_y(train_Set_Extracted, train_Set_Signals, train_Set_Codes,sequence_size=0, pad = pad, t=1)

Count:  1
Check shape:  (2500, 38)
2.3069684505462646
Number of off set cycles:  387
Cycles saved is lower than expected: amount  2474
Count:  18
Number of off set cycles:  1854
Cycles saved is lower than expected: amount  1007
Count:  30
Still the Cycles saved is lower than expected: amount  2014
Count:  30
Number of off set cycles:  131
Count:  50
Check shape:  (2500, 38)
310.25015139579773
Number of off set cycles:  900
Cycles saved is lower than expected: amount  1961
Count:  58
Number of off set cycles:  826
Cycles saved is lower than expected: amount  2035
Count:  69
Number of off set cycles:  1
Number of off set cycles:  2
Number of off set cycles:  11
Number of off set cycles:  48
Count:  100
Check shape:  (2500, 38)
1023.454431772232
Number of off set cycles:  12
Number of off set cycles:  18
Number of off set cycles:  79
Number of off set cycles:  208
Number of off set cycles:  200
Number of off set cycles:  147
Number of off set cycles:  298
Count:  150
Check shape:  (2500, 

In [56]:
np.save("../Datasets/Cycle Signals/y_Train_Program_2_No_Offset.npy", np.array(y_Train_Program_2))

In [57]:
y_Train_Program_3 = extractInformationFromTrainingSets_y(train_Set_Extracted, train_Set_Signals, train_Set_Codes,sequence_size=0, pad = pad, t=2)

Count:  1
Check shape:  (2500, 38)
12.627126932144165
Number of off set cycles:  335
Count:  50
Check shape:  (2500, 38)
331.96250462532043
Number of off set cycles:  403
Number of off set cycles:  57
Number of off set cycles:  6
Number of off set cycles:  1
Number of off set cycles:  219
Number of off set cycles:  709
Cycles saved is lower than expected: amount  2469
Count:  80
Number of off set cycles:  279
Number of off set cycles:  1138
Cycles saved is lower than expected: amount  2040
Count:  87
Count:  100
Check shape:  (2500, 38)
1101.2458312511444
Number of off set cycles:  424
Number of off set cycles:  57
Number of off set cycles:  151
Number of off set cycles:  436
Count:  150
Check shape:  (2500, 38)
2308.991327524185
Cycles saved is lower than expected: amount  2274
Count:  173
174


In [58]:
np.save("../Datasets/Cycle Signals/y_Train_Program_3_No_Offset.npy", np.array(y_Train_Program_3))

In [59]:
y_Train_Program_4 = extractInformationFromTrainingSets_y(train_Set_Extracted, train_Set_Signals, train_Set_Codes,sequence_size=0, pad = pad, t=3)

Count:  1
Check shape:  (2500, 38)
3.966050863265991
Number of off set cycles:  318
Number of off set cycles:  138
Count:  50
Check shape:  (2500, 38)
296.38065695762634
Number of off set cycles:  236
Number of off set cycles:  7
Number of off set cycles:  39
Number of off set cycles:  114
Count:  100
Check shape:  (2500, 38)
1038.877997636795
Number of off set cycles:  423
Number of off set cycles:  161
Number of off set cycles:  79
Number of off set cycles:  215
135


In [60]:
np.save("../Datasets/Cycle Signals/y_Train_Program_4_No_Offset.npy", np.array(y_Train_Program_4))